# Application of chosen model on whole dataset

In short, this didn't work. The dataset seems to be too big for me to be able to process it. Even when I tried breaking the code into small batches to make the needed matrix (part that I was never able to bypass to then get to the training part of the model) somewhat easier to process for the computer, it still failed.

## Imports and setup

In [1]:
# Importing libraries
import pandas as pd
import numpy as np
from collections import Counter
import pickle

import nltk
from nltk.probability import FreqDist
from nltk.classify.scikitlearn import SklearnClassifier
from nltk.metrics import ConfusionMatrix

from sklearn.linear_model import LogisticRegression

# My own functions
from NLP_Functions import find_features, make_matrix, clean_up, tokenize, stem_and_lemmatize, remove_stopwords

In [2]:
# Importing the dataset
#df = pd.read_csv('Datasets/games_comments_cleaned.csv', index_col = 0)

# Importing the dataset already with the comments processed and target created
## I ran this before and then exported to be safe 'cause I had kernel problems
df = pd.read_json('Datasets/comments_processed.json')  # skip straight to vars initialization

In [14]:
# Since the kernel keeps dying with the whole comments dataset, I will get half of it and try with that
df_cut = df.sample(df.shape[0]//2, random_state = 18)

df_cut.reset_index(drop = True, inplace = True)

# Exporting this half DF just in case
df_cut.to_json('Datasets/comments_processed_half.json')

df_cut.head()

,Title,Platform,Userscore,Comment,Comments_Processed,Username,Target,Target_NPS
0,Super Smash Bros. for Wii U,WiiU,9,"This game is amazing, improves in every aspect...","[thi, game, amaz, improv, everi, aspect, previ...",Zin_49,Positive,Positive
1,Zero Escape: Zero Time Dilemma,3DS,8,A bunch of bootleg robotic versions of the mai...,"[bunch, bootleg, robot, version, main, charact...",Techbane,Negative,Neutral
2,Metro: Last Light,PC,6,I liked the original this is some innovation b...,"[like, origin, thi, innov, silent, main, chara...",gstiker5,Negative,Negative
3,PlanetSide 2,PC,5,Planetside 2 has battles of epic scale and de...,"[planetsid, ha, battl, epic, scale, decent, gf...",NasseSeta,Negative,Negative
4,Diablo III,PC,1,Game looks good and overall i like the graphic...,"[game, look, good, overal, like, graphic, game...",JamesLFranco,Negative,Negative


# Preparing the dataset

## Creating 'Target' column

I will be using the 'Target' column, but I decided to create the Target_NPS one as well just to see how different they would be in the total dataset.

In [ ]:
# Creating the 'Target' column
## 9-10 positive
## 8 and below negative
df['Target'] = np.where((df['Userscore'] <= 8), 'Negative', 'Positive')

# Creating the 'Target_NPS' column
## NPS Scale - https://en.wikipedia.org/wiki/Net_Promoter
## 9-10 = positive
## 7-8 = neutral
## 0-6 = negative
df['Target_NPS'] = np.where((df['Userscore'] <= 6), 'Negative', 'Positive')
df['Target_NPS'] = np.where(((df['Userscore'] >= 7) & (df['Userscore'] <= 8)), 'Neutral', df['Target_NPS'])

# Checking the different proportion of values
## unsure if I should balance these or not
print(df['Target'].value_counts())
df['Target_NPS'].value_counts()

## Processing the comments

In [ ]:
# Creating the 'Comments_Processed' column
df['Comments_Processed'] = df['Comment'].apply(lambda x: stem_and_lemmatize(remove_stopwords(tokenize(clean_up(x)))))
## this took 25 mins to run...

In [ ]:
# Rearranging the columns order to be prettier
cols = df.columns.tolist()
df = df[['Title', 'Platform', 'Userscore', 'Comment', 'Comments_Processed', 'Username', 'Target', 'Target_NPS']]
df.head()

In [ ]:
# Exporting the DF now just to be safe!
df.to_json('Datasets/comments_processed.json')

In [15]:
df_cut.head()

,Title,Platform,Userscore,Comment,Comments_Processed,Username,Target,Target_NPS
0,Super Smash Bros. for Wii U,WiiU,9,"This game is amazing, improves in every aspect...","[thi, game, amaz, improv, everi, aspect, previ...",Zin_49,Positive,Positive
1,Zero Escape: Zero Time Dilemma,3DS,8,A bunch of bootleg robotic versions of the mai...,"[bunch, bootleg, robot, version, main, charact...",Techbane,Negative,Neutral
2,Metro: Last Light,PC,6,I liked the original this is some innovation b...,"[like, origin, thi, innov, silent, main, chara...",gstiker5,Negative,Negative
3,PlanetSide 2,PC,5,Planetside 2 has battles of epic scale and de...,"[planetsid, ha, battl, epic, scale, decent, gf...",NasseSeta,Negative,Negative
4,Diablo III,PC,1,Game looks good and overall i like the graphic...,"[game, look, good, overal, like, graphic, game...",JamesLFranco,Negative,Negative


## Initializing the basic variables for the model

In [16]:
# Creating the bag of words
bow = [word for lst in df_cut['Comments_Processed'] for word in lst]
fdist = FreqDist(bow)

# Getting just the 10k most common words
most_common = fdist.most_common(10000)  # wonder if I should change this value around or not (trying w/ 10k this time)

In [18]:
# TRYING TO BREAK IT DOWN IN SMALLER PIECES SO THE KERNEL DOESN'T DIE
## Nope, kernel still died with the whole DF
## WORKED WITH HALF THE DATASET BUT KERNEL BECAME UNRESPONSIVE AFTERWARDS...
n_batches = 20
batch_size = df_cut.shape[0]//n_batches
batches = []

# Couldn't make the range start from 0, so it starts from 1. At least it's just 1 row that I lose in 282k...
batches = [make_matrix(df_cut.loc[(n+1)*batch_size - batch_size +1:n*batch_size+batch_size, 'Comment'], 
                       df_cut.loc[(n+1)*batch_size - batch_size +1:n*batch_size+batch_size, 'Target'], most_common) 
          for n in range(n_batches)]

# Flattening the batches
batches_flat = [item for lst in batches for item in lst]  # this is how it needs to be in the end

In [ ]:
# EXPORTING/PICKLING THIS TOO! TAKING NO CHANCES LOL.
## seems to have saved an empty or broken file...
save_batches_flat = open('batches_flat_df_cut.pickle', 'wb')
pickle.dump(batches_flat, save_batches_flat)
save_batches_flat.close()

In [ ]:
len(batches_flat)

In [ ]:
'''
Initial for loop I had before turning it into a list comprehension
for n in range(n_batches):
    batches.append(make_matrix(df.loc[(n+1)*batch_size - batch_size +1:n*batch_size+batch_size, 'Comment'], 
                               df.loc[(n+1)*batch_size - batch_size +1:n*batch_size+batch_size, 'Target'], most_common))
'''

In [ ]:
'''FOR THE TARGET COLUMN'''
# Building the features and making the matrix
#matrix = make_matrix(df['Comment'], df['Target'], most_common)  # this is something that takes forever...

# Defining the size to use for the training and testing
size = int(len(batches_flat) * 0.20)

# Training with 80% of the data and testing against the remaining 20%
training_set = batches_flat[size:]
testing_set = batches_flat[:size]

## Logistic Regression Model Application

In [ ]:
# TESTING WITH TARGET
LogisticRegression_classifier = SklearnClassifier(LogisticRegression(solver = 'saga', n_jobs = -1))
LogisticRegression_classifier.train(training_set)


print('Logistic Regression accuracy (Target):', 
      str(round(nltk.classify.accuracy(LogisticRegression_classifier, testing_set) * 100, 2)) + '%')

## Confusion Matrix

In [ ]:
# Predictions of the testing_set with the Target column
pred_list_LR = [LogisticRegression_classifier.classify(testing_set[i][0]) for i in range(len(testing_set))]
print(pred_list_LR.count('Positive'))  
print(pred_list_LR.count('Negative'))  

In [ ]:
# Unpacking the references (Positive or Negative) of the testing set to use for the confusion matrix
ref = [testing_set[i][1] for i in range(len(testing_set))]  # pos: xx | neg: xx

# List of predictions ran above
tagged = pred_list_LR

# The actual confusion matrix
cm = ConfusionMatrix(ref, tagged)

print(cm)
cm

labels = set('Positive Negative'.split())

true_positives = Counter()
false_negatives = Counter()
false_positives = Counter()

for i in labels:
    for j in labels:
        if i == j:
            true_positives[i] += cm[i, j]
        else:
            false_negatives[i] += cm[i, j]
            false_positives[j] += cm[i, j]

# This seems to be the best model as the accuracy already predicted
print('TP + TN:', sum(true_positives.values()), true_positives)
print('FP + FN:', sum(false_positives.values()), false_positives)

## Function to Predict a Review's Label

In [ ]:
def predictor(text):
    prediction = find_features(text, most_common)
    return print('Prediction:', LogisticRegression_classifier.classify(prediction))

predictor('This game is amazing!')
predictor('This game is terrible.')

## Exporting Files for Future Use

In [ ]:
# Saving the 10k most_common words of this whole DF
save_most_common = open('10k_most_common_final.pickle', 'wb')
pickle.dump(most_common, save_most_common)
save_most_common.close()

In [ ]:
# Saving the trained LR algorithm
save_classifier = open('Logistic_Regression_final.pickle', 'wb')
pickle.dump(LogisticRegression_classifier, save_classifier)
save_classifier.close()